<a href="https://colab.research.google.com/github/aikokass82-design/home-tasks-/blob/main/Lesson_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount("/content/drive")
from datetime import datetime

In [ ]:
advertising_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reclamni_zosobi-info.csv")
advertising_info

In [ ]:
distributor_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/distributor-info.csv")
distributor_info

In [ ]:
#1 zadaniye
advertising_info.columns = advertising_info.columns.str.strip()
distributor_info.columns = distributor_info.columns.str.strip()

advertising_info.dropna(subset=["id", "distributorid"], inplace=True)
distributor_info.dropna(subset=["id"], inplace=True)

advertising_info.drop_duplicates(inplace=True)
distributor_info.drop_duplicates(inplace=True)

advertising_info['issuedDate'] = pd.to_datetime(advertising_info['issuedDate'], errors='coerce')
advertising_info['validFrom'] = pd.to_datetime(advertising_info['validFrom'], errors='coerce')
advertising_info['validThrough'] = pd.to_datetime(advertising_info['validThrough'], errors='coerce')
advertising_info['status'] = advertising_info['status'].str.lower().str.strip()
advertising_info['distributorid'] = advertising_info['distributorid'].astype(str)
distributor_info['id'] = distributor_info['id'].astype(str)

print(advertising_info.info())
print(distributor_info.info())

In [ ]:
today = pd.Timestamp.today()
advertising_info["isActiveNow"] = (advertising_info["validFrom"]<=today)&(advertising_info["validThrough"]>=today)
print(advertising_info[["id", "validFrom","validThrough", "isActiveNow"]].head(10))
print(advertising_info["isActiveNow"].value_counts())

In [ ]:
!pip install ydata-profiling

In [ ]:
import ydata_profiling
profile_adv = ydata_profiling.ProfileReport(advertising_info)
profile_adv.to_file("advertising_info_report.html")

profile_dist = ydata_profiling.ProfileReport(distributor_info)
profile_dist.to_file("distributor_info_report.html")

2 zadaniye

In [ ]:
#1
type_counts = advertising_info["type"].value_counts()
print(type_counts.head(5))

In [ ]:
#2
pivo_type_district = advertising_info.pivot_table(index="district", columns="type", values="id", aggfunc="count", fill_value=0)
print(pivo_type_district)

In [ ]:
#3
advertising_info = advertising_info.merge(distributor_info, left_on='distributorid', right_on='id', how='left')
advertising_info.rename(columns={'distributor': 'company'}, inplace=True)
company_counts = advertising_info['company'].value_counts()
print(company_counts)

In [ ]:
#4
company_area = advertising_info.groupby("company")["area"].sum().sort_values(ascending=False)
print(company_area.head(10))

In [ ]:
#5
pivot_area_district = advertising_info.pivot_table(index="district", columns="company", values="area", aggfunc="sum", fill_value=0)
print(pivot_area_district)

In [ ]:
#6
today=pd.Timestamp.today()
closest_expiry = advertising_info.sort_values("validThrough").head(10)
print(closest_expiry[["company", "validThrough"]])

In [ ]:
#7
company_counts = advertising_info["company"].value_counts()
top5_companies = company_counts.head(5).index
top5_data=advertising_info[advertising_info["company"].isin(top5_companies)]
pivot_top5_type = top5_data.pivot_table(index="company", columns="type", aggfunc="size", fill_value=0)
print(pivot_top5_type)

In [ ]:
top_type_per_company = pivot_top5_type.idxmax(axis=1)
print("Наиболее предпочтительный тип рекламы у топ5 компаний:")
print(top_type_per_company)